## <span style="color:grey">Initialization</span>

In [1]:
try:
    mode
except Exception as e:
    %run "KiteConnect_Library.ipynb"  
    logging.debug(e)
    mode = "Algo"
    papertrade = True

tradelogger.setLevel(logging.DEBUG)

In [2]:
tradelogger.info("Let's do trading")

##  <span style="color:green">Trading Strategy </span>

In [3]:
from talib import MIN, MAX, CDLCLOSINGMARUBOZU,CDL3WHITESOLDIERS,CDLMORNINGSTAR,CDLMORNINGDOJISTAR,CDL3LINESTRIKE,CDL3OUTSIDE,CDLENGULFING,CDLBELTHOLD,CDLABANDONEDBABY,CDL3INSIDE,CDLPIERCING,CDLBREAKAWAY,CDLXSIDEGAP3METHODS,CDLHAMMER,CDLMARUBOZU,CDL3BLACKCROWS,CDLIDENTICAL3CROWS,CDLEVENINGSTAR,CDLEVENINGDOJISTAR,CDLDARKCLOUDCOVER,CDLSHOOTINGSTAR,CDLHANGINGMAN,CDLHARAMI,CDLHARAMICROSS,CDLINVERTEDHAMMER,CDLSPINNINGTOP,CDLGRAVESTONEDOJI,CDLDOJI,CDLDOJISTAR,CDLDRAGONFLYDOJI
cdlfunc_array = [CDLCLOSINGMARUBOZU,CDL3WHITESOLDIERS,CDLMORNINGSTAR,CDLMORNINGDOJISTAR,CDL3LINESTRIKE,CDL3OUTSIDE,CDLENGULFING,CDLBELTHOLD,CDLABANDONEDBABY,CDL3INSIDE,CDLPIERCING,CDLBREAKAWAY,CDLXSIDEGAP3METHODS,CDLHAMMER,CDLMARUBOZU,CDL3BLACKCROWS,CDLIDENTICAL3CROWS,CDLEVENINGSTAR,CDLEVENINGDOJISTAR,CDLDARKCLOUDCOVER,CDLSHOOTINGSTAR,CDLHANGINGMAN,CDLHARAMI,CDLHARAMICROSS,CDLINVERTEDHAMMER,CDLSPINNINGTOP,CDLGRAVESTONEDOJI,CDLDOJI,CDLDOJISTAR,CDLDRAGONFLYDOJI]

def detectCDPattern(prices, noofcandles=15, strPlot="engulfing"):
    #https://github.com/mrjbq7/ta-lib
    
    annotateText = []
    annotateIndex = []
    
    for index, curr in prices.iloc[noofcandles:].iterrows():
        text = ""
        
        index_r = prices.index.get_loc(index)+1
        index_l = max(0, index_r - noofcandles)
        #rice = pd.DataFrame()
        popen = prices.iloc[index_l:index_r]['open']
        phigh = prices.iloc[index_l:index_r]['high']
        pclose = prices.iloc[index_l:index_r]['close']
        plow = prices.iloc[index_l:index_r]['low']
        
        function = CDLDOJI
        
        for cdlfunc in cdlfunc_array:
            ret_val = cdlfunc(popen, phigh, plow, pclose)[-1]
            if (ret_val > 0):
                text += "Bull: "+re.sub('CDL', '', cdlfunc.__name__)+"<br>"
            elif (ret_val < 0):
                text += "Sell: "+re.sub('CDL', '', cdlfunc.__name__)+"<br>"
                
        if text != "":
            annotateText.append(text)
            annotateIndex.append(index)
            
            a.appendHoverText(index,text)
            #logger.debug(index.strftime("%d-%m-%y %H:%M")+": "+text)
    


In [54]:
# ====== Tradescript Wrapper =======
# Methods
REF = lambda df, i: df.shift(i)
#TREND_UP = lambda a,b: ROC(a, b) >= 0.1
#TREND_DOWN = lambda a,b: ROC(a, b) <= -0.1

import traceback
TREND_UP = lambda a,b: a > MAX(REF(a,1),b)
TREND_DOWN = lambda a,b: a < MIN(REF(a,1),b)

CROSSOVER = lambda a, b: (REF(a,1)<=REF(b,1)) & (a > b)
isblackout = True

blackoutFrom = '11:30:00'
blackoutTo = '12:30:00'

blackout = lambda a,x,y=blackoutFrom,z=blackoutTo: a[isblackout & (a.index >= x) & (a.index < x+' '+y) | (a.index > x+' '+z)]


class algoTrade:
   
    def __init__(s, price):
        #logging.debug(price.tail(10))
        s.OPEN = price['open']
        s.CLOSE = price['close']
        s.HIGH = price['high']
        s.LOW = price['low']
        
        
        s.haOPEN  = (s.OPEN.shift(1) + s.CLOSE.shift(1))/2
        s.haHIGH  = pd.DataFrame([s.HIGH,s.OPEN,s.CLOSE]).max(axis = 0, skipna = True)
        s.haLOW   = pd.DataFrame([s.HIGH,s.LOW,s.CLOSE]).min(axis = 0, skipna = True)
        s.haCLOSE = (s.OPEN+s.HIGH+s.LOW+s.CLOSE)/4
        
        s.BBT, s.BBM, s.BBB = BBANDS( s.CLOSE, 20,2,2,0)
        #s.BBT2, s.BBM2, s.BBB2 = BBANDS( s.CLOSE, 20,3,3,0)
        s.macd, s.macdsignal, s.macdhist = MACDEXT(s.CLOSE, fastperiod=12, slowperiod=26, signalperiod=9,  fastmatype=1, slowmatype=1,signalmatype=1)
        s.SD = STDDEV(s.CLOSE)
        s.fastk, s.fastd = STOCHF(s.HIGH, s.LOW, s.CLOSE)
        s.rsi = RSI(s.CLOSE, timeperiod=20)
        s.STATE = ""
        s.profit = 0
        s.stoploss = 0
        s.openpositions = 0
        s.target = 0
        s.min = MIN(s.CLOSE, timeperiod=30)
        s.max = MAX(s.CLOSE, timeperiod=30)
        
        
        
        s.hovertextarray = pd.DataFrame(np.full(s.CLOSE.index.shape[0],""))
    
    def getHoverText(s,index):
        iloc = a.CLOSE.index.get_loc(index)
        return s.hovertextarray.iloc[iloc][0]
        
    def appendHoverText(s, index,text):
        #logger.debug(str(index)+": "+text)
        iloc = a.CLOSE.index.get_loc(index)
        s.hovertextarray.iloc[iloc] = s.hovertextarray.iloc[iloc] + text
    
    # Long Strategies
    def long_bb(s):
        temp_df = 0
        #temp_df = temp_df | (REF(s.fastk,1) <=20)&(s.fastk>20)
        #temp_df = temp_df | CROSSOVER(s.fastk, s.fastd)
        #temp_df = temp_df | (REF(s.rsi,1) <=25)
        #temp_df = temp_df | (REF(s.macd,1) <=0)&(s.macd>0)
        #temp_df = temp_df | CROSSOVER(s.macdsignal, s.macd)
        #temp_df = temp_df | (s.CLOSE.shift(1) == s.min.shift(1)) & (s.CLOSE > s.OPEN)
        temp_df = temp_df & (s.CLOSE.shift(1) == s.min.shift(1)) & (s.CLOSE > s.OPEN)  
        
        return pd.DataFrame(  temp_df, columns=["buy"] )
     
    # Short Strategies
    def short_bb(s):
        temp_df = 0
        #temp_df = temp_df | (REF(s.fastk,1) >=80)&(s.fastk<80)
        #temp_df = temp_df | (REF(s.rsi,1) >=75)
        #temp_df = temp_df | CROSSOVER(s.macd, s.macdsignal)
        #temp_df = temp_df | CROSSOVER(s.BBT, s.OPEN)
        #temp_df = temp_df | (REF(s.macd,1) >=0)&(s.macd<0)
        temp_df = temp_df & (s.CLOSE.shift(1) == s.max.shift(1)) & (s.CLOSE < s.OPEN)
        return pd.DataFrame( temp_df, columns=["sell"])
    
    
    def long_pa(s):
        temp_df = False
        temp_df = temp_df | (CDLMORNINGSTAR(s.OPEN, s.HIGH, s.LOW, s.CLOSE) == 100)
        temp_df = temp_df | (CDLMORNINGDOJISTAR(s.OPEN, s.HIGH, s.LOW, s.CLOSE) == 100)
        temp_df = temp_df | (CDL3WHITESOLDIERS(s.OPEN, s.HIGH, s.LOW, s.CLOSE) == 100)
        
        temp_df = temp_df | (CDLENGULFING(s.OPEN, s.HIGH, s.LOW, s.CLOSE) == 100)
        temp_df = temp_df | (CDL3LINESTRIKE(s.OPEN, s.HIGH, s.LOW, s.CLOSE) == 100)
        temp_df = temp_df | (CDL3OUTSIDE(s.OPEN, s.HIGH, s.LOW, s.CLOSE) == 100)
        temp_df = temp_df | (CDLABANDONEDBABY(s.OPEN, s.HIGH, s.LOW, s.CLOSE) == 100)
        temp_df = temp_df | ((CDLBELTHOLD(s.OPEN, s.HIGH, s.LOW, s.CLOSE) == 100) & TREND_DOWN(s.CLOSE,4))
        
        temp_df = temp_df | CDL3INSIDE(s.OPEN, s.HIGH, s.LOW, s.CLOSE) == 100
        temp_df = temp_df | (CDLHARAMI(s.OPEN, s.HIGH, s.LOW, s.CLOSE) == 100)
        temp_df = temp_df | (CDLPIERCING(s.OPEN, s.HIGH, s.LOW, s.CLOSE) == 100)
        temp_df = temp_df | (CDLHAMMER(s.OPEN, s.HIGH, s.LOW, s.CLOSE) == 100)
        temp_df = temp_df | (CDLDOJISTAR(s.OPEN, s.HIGH, s.LOW, s.CLOSE) != 0)
        
        temp_df = temp_df | (CDLMARUBOZU(s.OPEN, s.HIGH, s.LOW, s.CLOSE) == 100) 
        temp_df = temp_df | (CDLCLOSINGMARUBOZU(s.OPEN, s.HIGH, s.LOW, s.CLOSE) ==100)

        
        temp_df = TREND_DOWN(s.OPEN,5) & temp_df
        
        return pd.DataFrame(temp_df, columns=["buy"])
    def short_pa(s):
        temp_df = False
        
        temp_df = temp_df | (CDL3BLACKCROWS(s.OPEN, s.HIGH, s.LOW, s.CLOSE) == -100)
        temp_df = temp_df | (CDLIDENTICAL3CROWS(s.OPEN, s.HIGH, s.LOW, s.CLOSE) == -100)
        temp_df = temp_df | (CDLEVENINGSTAR(s.OPEN, s.HIGH, s.LOW, s.CLOSE) == -100)
        temp_df = temp_df | (CDL3LINESTRIKE(s.OPEN, s.HIGH, s.LOW, s.CLOSE) == -100)
        
        temp_df = temp_df | (CDLENGULFING(s.OPEN, s.HIGH, s.LOW, s.CLOSE) == -100)
        temp_df = temp_df | (CDLEVENINGDOJISTAR(s.OPEN, s.HIGH, s.LOW, s.CLOSE) == -100)
        temp_df = temp_df | (CDL3OUTSIDE(s.OPEN, s.HIGH, s.LOW, s.CLOSE) == -100)
        temp_df = temp_df | (CDLBELTHOLD(s.OPEN, s.HIGH, s.LOW, s.CLOSE) == -100)
        temp_df = temp_df | (CDLABANDONEDBABY(s.OPEN, s.HIGH, s.LOW, s.CLOSE) == -100)
        temp_df = temp_df | (CDLHARAMI(s.OPEN, s.HIGH, s.LOW, s.CLOSE) == -100)
        temp_df = temp_df | (CDLDARKCLOUDCOVER(s.OPEN, s.HIGH, s.LOW, s.CLOSE) == -100)
        temp_df = temp_df | (CDLDOJISTAR(s.OPEN, s.HIGH, s.LOW, s.CLOSE) != 0)
        temp_df = temp_df | (CDLHANGINGMAN(s.OPEN, s.HIGH, s.LOW, s.CLOSE) == -100)
        temp_df = temp_df | (CDLEVENINGSTAR(s.OPEN, s.HIGH, s.LOW, s.CLOSE) == -100) 
        temp_df = temp_df | (CDLEVENINGDOJISTAR(s.OPEN, s.HIGH, s.LOW, s.CLOSE) == -100)
        temp_df = temp_df | (CDLMARUBOZU(s.OPEN, s.HIGH, s.LOW, s.CLOSE) == -100)
        temp_df = temp_df | (CDLCLOSINGMARUBOZU(s.OPEN, s.HIGH, s.LOW, s.CLOSE) == -100)
        temp_df = temp_df | (CDLBELTHOLD(s.OPEN, s.HIGH, s.LOW, s.CLOSE) == -100)
        
        temp_df = TREND_UP(s.CLOSE,5) & temp_df
        
        return pd.DataFrame( temp_df , columns=["sell"])
    
    def long_breakout(s):
        temp_df = s.haCLOSE >= s.BBT.shift(1)
        temp_df = temp_df | (s.haCLOSE >= s.haOPEN.shift(2))
        temp_df = temp_df & ( CROSSOVER(s.OPEN, s.BBT) | CROSSOVER(s.OPEN, s.BBB) | CROSSOVER(s.OPEN, s.BBM))
        return pd.DataFrame( temp_df , columns=["buy"])
    
    def short_breakout(s):
        temp_df = s.haOPEN <= s.BBB.shift(1)
        temp_df = temp_df | (s.haCLOSE <= s.haOPEN.shift(2))
        temp_df = temp_df & ( CROSSOVER(s.BBT, s.OPEN) | CROSSOVER(s.BBB, s.OPEN) | CROSSOVER(s.BBM, s.OPEN))
        return pd.DataFrame( temp_df , columns=["sell"])
    
    def long_ha(s):
        temp_df = (s.haCLOSE >= REF(s.haOPEN,1)) & (s.haCLOSE > s.haOPEN) #& (REF(s.CLOSE,1) < REF(s.OPEN,1))
        return pd.DataFrame( temp_df , columns=["buy"])
    
    def short_ha(s):
        temp_df = (s.haCLOSE <= REF(s.haOPEN,1)) & (s.haOPEN > s.haCLOSE) #& (REF(s.CLOSE,1) > REF(s.OPEN,1))
        return pd.DataFrame( temp_df , columns=["sell"])

    def tradeDecision(s):
        global isblackout
        #isblackout = True
        #s.buy  = s.long_ha()# & s.long_bb()
        #s.sell = s.short_ha()# & s.short_bb()
        
        #print("tradedecision")
        
        s.buy  = s.long_pa()
        s.sell = s.short_pa()
        
#        s.buy = s.long_breakout()
#        s.sell = s.short_breakout()
        
        s.buy['low'] = s.LOW
        s.sell['low'] = s.LOW
        s.buy['high'] = s.HIGH
        s.sell['high'] = s.HIGH
        s.buy['close'] = -1 * s.CLOSE
        s.sell['close'] = s.CLOSE

        if s.buy['buy'].any():
            s.buy = blackout(s.buy,s.buy.index[-1].date().strftime("%Y-%m-%d"))
        if s.sell['sell'].any():
            s.sell = blackout(s.sell, s.sell.index[-1].date().strftime("%Y-%m-%d"))
        
        
        s.buy = s.buy[s.buy['buy']]
        s.sell = s.sell[s.sell['sell']]
        
        s.buysell = pd.concat([s.buy.close, s.sell.close]).sort_index()
        if not s.buysell.empty:
            s.buysell = s.buysell[s.buysell.index>s.buysell.index[-1].date().strftime("%Y-%m-%d")]
        
        #logging.debug(s.buysell)
        return (s.buy,s.sell)
    
    def payoff(s):
        global isblackout
        payoff_df = pd.DataFrame(s.buysell)
        openpositions = 0
        squareoff = True
        price = 0
        
        logTrade = lambda prefix, i: logger.debug(prefix+i.strftime("[%H:%M]")+"\t"+str(openpositions)+"\t"+str(round(s.profit[0],2))+"\t\t"+str(price)+"\t | "+ re.sub('<br>',', ',s.getHoverText(i),0))
        
        morningCutOff = s.CLOSE.index[-1].strftime("%Y-%m-%d 11:30:00")
        
        logger.debug("Trade\tQty\tProfit\t\tClose Price")
        
        for i, p in payoff_df.iterrows():
            if squareoff and isblackout and i >= payoff_df.index[-1].replace(hour=11, minute=30, second=0, microsecond=0):
                price = a.CLOSE.loc[morningCutOff]
                if openpositions > 0:
                    s.profit = s.profit + price
                    openpositions = 0
                    logTrade("SELL[11:30]:",i)
                elif  openpositions < 0:
                    s.profit = s.profit - price
                    openpositions = 0
                    logTrade("BUY [11:30]:",i)
                    
                s.STATE = ""
                squareoff = False
                
            if not squareoff and  i <= payoff_df.index[-1].replace(hour=12, minute=30, second=0, microsecond=0):
                continue
            
            
            #TODO: Stoploss and target squareoff
            
                
            if s.STATE != "SELL" and p.values < 0:
                s.STATE = "SELL"
                price = p.values[0]
                s.profit = s.profit + p.values
                openpositions = openpositions + 1
                logTrade("BUY :",i)
            if s.STATE != "BUY" and p.values > 0:
                s.STATE = "BUY"
                price = p.values[0]
                s.profit = s.profit + p.values
                openpositions = openpositions - 1
                logTrade("SELL:",i)
         
        if isblackout:
            close = a.CLOSE.loc[s.CLOSE.index[-1].strftime("%Y-%m-%d 15:00:00")]
        else:
            close = a.CLOSE[-1]
        
        if openpositions > 0:
            s.profit = s.profit + close
            openpositions = 0
            price = close
            logTrade("SELL:[15:00]",i)
        elif openpositions < 0:
            s.profit = s.profit - close
            openpositions = 0
            price = close
            logTrade("BUY :[15:00]",i)
            
        return(s.profit)
    
    def plot(s, symbol, noofcandles=120, ha=False):
        init_notebook_mode(connected=True)
        fig = tools.make_subplots(rows=4, cols=1, shared_xaxes=True, row_width=[1,1,1,3], vertical_spacing = 0.05)
        fig['layout']['xaxis'] = dict(rangeslider = dict(visible=False), side="bottom") #, range=[xMin,xMax])
        fig['layout'].update(height=750, plot_bgcolor='rgba(0,0,0,0)', title="Charts for "+symbol)
        
        fig['layout']['yaxis']['anchor'] = 'x'
        fig['layout']['yaxis']['side'] = 'right'

        fig['layout']['xaxis']['rangeselector'] = dict(
                    buttons=list([dict(count=1, label='1h', step='hour', stepmode='backward'),
                                  dict(count=2, label='2h', step='hour', stepmode='backward'),
                                  dict(count=3, label='3h', step='hour', stepmode='backward'),
                                  dict(count=6, label='4h', step='hour', stepmode='backward'),
                                  dict(count=6, label='1d', step='hour', stepmode='backward'),
                                  dict(step='all')]))
        
        yMin = s.LOW.iloc[-1*noofcandles:-1].min()*0.99
        yMax = s.HIGH.iloc[-1*noofcandles:-1].max()*1.001

        xMin = s.CLOSE.index[-1*noofcandles]
        xMax = s.CLOSE.index[-1]
        fig['layout']['yaxis']['range'] = [yMin, yMax]
        fig['layout']['xaxis']['range'] = [xMin, xMax]
        
        #print(s.buy.shape)
        #print(s.sell.shape)
        
        
        #print("plot")
        
        traceBuy=go.Scatter(x=s.buy.index.astype('str'), y=-1 * s.buy.close, name='BUY', mode="markers",marker = dict(color = 'rgba(119, 221, 119, 0.2)', size = 25, symbol='circle', line = dict(color = 'rgb(119, 221, 119)', width = 3)),showlegend=False)
        
        traceSell=go.Scatter(x=s.sell.index.astype('str'), y=s.sell.close, name='SELL', mode="markers",marker = dict(color = 'rgba(255, 0, 0,0.2)',size = 25, symbol='circle', line = dict(color = 'rgb(255, 0, 0)', width = 3)),showlegend=False)
       
        if ha:
            trace = go.Candlestick(x=s.CLOSE.index.astype('str'), open=s.haOPEN, high=s.haHIGH, low=s.haLOW, close=s.haCLOSE, name="Heikin Ashi", showlegend=False, hoverinfo = 'x+y+text', hovertext=s.hovertextarray)        
        else:
            trace = go.Candlestick(x=s.CLOSE.index.astype('str'), open=s.OPEN, high=s.HIGH, low=s.LOW, close=s.CLOSE, name="Candlestick", showlegend=False, hoverinfo = 'x+y+text', hovertext=s.hovertextarray)      
            
        traceMACD = go.Scatter(x=s.CLOSE.index.astype('str'), y=s.macd, name='MACD', line=dict(color='black'),showlegend=False)
        traceMACDSignal = go.Scatter(x=s.CLOSE.index.astype('str'), y=s.macdsignal, name='MACD signal', line=dict(color='red'),showlegend=False)
        traceMACDHist = go.Bar(x=s.CLOSE.index.astype('str'), y=s.macdhist, name='MACD Hist', marker=dict(color="grey"),showlegend=False)
        traceSK  = go.Scatter(x=s.CLOSE.index.astype('str'), y=s.fastk, name='%K', line=dict(color='black'), yaxis='y3',showlegend=False)
        traceSD  = go.Scatter(x=s.CLOSE.index.astype('str'), y=s.fastd, name='%D', line=dict(color='red'),showlegend=False)
        traceBBT = go.Scatter(x=s.BBT.index.astype('str'), y=s.BBT, name='BB_up',  line=dict(color='lightgrey'),showlegend=False, hoverinfo = 'none')
        traceBBB = go.Scatter(x=s.BBB.index.astype('str'), y=s.BBB, name='BB_low',  line=dict(color='lightgrey'), fill = 'tonexty', fillcolor="rgba(0,40,100,0.02)",showlegend=False, hoverinfo = 'none')
        traceBBM = go.Scatter(x=s.BBM.index.astype('str'), y=s.BBM, name='BB_mid',  line=dict(color='lightgrey'), fill = 'tonexty', fillcolor="rgba(0,40,100,0.02)",showlegend=False, hoverinfo = 'none')
        
        traceMIN = go.Scatter(x=s.min.index.astype('str'), y=s.min, name='MIN',  line=dict(color='darkblue'), showlegend=False, hoverinfo = 'none')
        traceMAX = go.Scatter(x=s.max.index.astype('str'), y=s.max, name='MAX',  line=dict(color='darkcyan'), showlegend=False, hoverinfo = 'none')
        
        traceRSI = go.Scatter(x=s.rsi.index.astype('str'), y=s.rsi, name='RSI',  line=dict(color='darkcyan'), showlegend=False)
        
        fig.append_trace(traceBBT, 1, 1)
        fig.append_trace(traceBBB, 1, 1)
        fig.append_trace(traceBBM, 1, 1)
        fig.append_trace(traceBuy, 1, 1)
        fig.append_trace(traceSell, 1, 1)
        fig.append_trace(traceMIN, 1, 1)
        fig.append_trace(traceMAX, 1, 1)
    
        fig.append_trace(traceSK, 2, 1)
        fig.append_trace(traceSD, 2, 1)
        
        
        
        fig['layout']['yaxis2']['title']="Stochastics"
        fig['layout']['yaxis2']['side']="right"
        fig['layout']['yaxis2']['anchor']="x"
        fig['layout']['yaxis2']['tickvals']=[20,80,30,70]
        
        fig.append_trace(traceMACD, 3, 1)
        fig.append_trace(traceMACDSignal, 3, 1)
        fig.append_trace(traceMACDHist, 3, 1)
        fig.append_trace(trace, 1, 1)
        
        fig['layout']['yaxis3']['anchor']="x"
        fig['layout']['yaxis3']['side']="right"
        fig['layout']['yaxis3']['title']="MACD"
        fig['layout']['yaxis1']['title']="Candlestick"
        
        
        fig.append_trace(traceRSI, 4, 1)
        fig['layout']['yaxis4']['title']="RSI"
        fig['layout']['yaxis4']['side']="right"
        fig['layout']['yaxis4']['anchor']="x"
        fig['layout']['yaxis4']['tickvals']=[30,50,70]
        
        iplot(fig, filename="plot/"+x+".html")
        
    # do not set timestamp for live trading
    def tradeBot(s, symbol, timestamp=dt.datetime.now().replace(second=0, microsecond=0)):
        global isblackout
        #payoff_df = pd.DataFrame(s.buysell)
        s.openpositions = 0
        s.squareoff = True
        s.price = 0
        
        timeNow = timestamp        
        delT1 = timeNow - dt.timedelta(minutes = 1)
        delT2 = timeNow - dt.timedelta(minutes = 2)
        
        buy, sell = s.tradeDecision()
        
        tradelogger.debug("STATE:" + s.STATE)
        tradelogger.debug(buy)
        tradelogger.debug(sell)
        
        
        tradelogger.debug(timeNow)
        tradelogger.debug(delT1)
        tradelogger.debug(delT2)
        
        #atr = ATR(HIGH, LOW, CLOSE)[-1]
        toTick = lambda x,n=5: np.round((np.floor(x *100)+n-1)/n)*n/100
        s.atr = s.CLOSE[-1]*0.025

        price = s.CLOSE[-1]
        trigger = s.CLOSE[-1]
        sl = toTick(price * 0.001)
        target = toTick(price * 0.01)

        check_trade = lambda x, dt1, dt2: True if (x.shape[0] > 0) and ((x.index[-1]==dt1) or (x.index[-1]==dt2)) else False
        print_trade = lambda x: logging.debug(x) if (x.empty != True) else False
        
        logTrade = lambda prefix, i: logger.debug(prefix+i.strftime("[%H:%M]")+"\t"+str(openpositions)+"\t"+str(round(s.profit[0],2))+"\t\t"+str(price)+"\t | "+ re.sub('<br>',', ',s.getHoverText(i),0))
        
        morningCutOff = s.CLOSE.index[-1].strftime("%Y-%m-%d 11:30:00")
        
        logger.debug("Trade\tQty\tProfit\t\tClose Price")
        
        
        if False:
            if squareoff and isblackout and i >= payoff_df.index[-1].replace(hour=11, minute=30, second=0, microsecond=0):
                price = a.CLOSE.loc[morningCutOff]
                if openpositions > 0:
                    s.profit = s.profit + price
                    openpositions = 0
                    logTrade("SELL[11:30]:",i)
                elif  openpositions < 0:
                    s.profit = s.profit - price
                    openpositions = 0
                    logTrade("BUY [11:30]:",i)

                s.STATE = ""
                squareoff = False

            if not squareoff and  i <= payoff_df.index[-1].replace(hour=12, minute=30, second=0, microsecond=0):
                continue

                
        # BUY in BUY state, SELL in STATE state
        if check_trade(buy, delT1, delT2) and (STATE != "SELL"):            
            tradelogger.info("buy:"+str(buy.index[-1]))
            tradelogger.debug(s.openpositions)
            
            qty = 1
            ##### Order Placement #####
            s.orderid = buy_bo(symbol, price=price, trigger=trigger, stoploss=sl, target=target, quantity=qty)
            #buy_slm(symbol, price, stoploss, quantity=1)
            s.STATE = "SELL"
            s.openpositions = s.openpositions + 1
            return -1 * price * qty
            
        elif check_trade(sell, delT1, delT2) and (STATE!="BUY"):                
            tradelogger.info("sell:"+str(sell.index[-1]))
            tradelogger.debug(s.openpositions)
           
            qty = 1
            ##### Order Placement #####
            s.orderid = sell_bo(symbol, price=price, trigger=trigger, stoploss=sl, target=target, quantity=qty)
            #sell_slm(symbol, price, stoploss, quantity=1)
            s.STATE = "BUY"
            openpositions = openpositions - 1
            return price * qty
        
        return 0

if mode == "Algo":
    x = 'TATAMOTORS'
    portfolio = pd.DataFrame([x])
    toDate = dt.datetime(2019,4,27,15,15)
    #portfolio = pd.DataFrame(downloadlist)

if mode == "Algo":
    open("log/live_log.log", "w")
    
    fromDate =   getFromDate(toDate)
    #logger.setLevel(logging.DEBUG)
    papertrade = True
    
    logging.basicConfig(filemode='w')
    cumsum = 0
    for i,y in portfolio.iterrows():
        logger.info(y[0])
    
        temp_data = getData(y[0], fromDate, toDate, "minute",force =True)

        a = algoTrade(temp_data)
        
        if portfolio.shape[0] == 1:
            detectCDPattern(temp_data)

        buy, sell = a.tradeDecision()

        
        try:
            payoff = round(a.payoff()[0],2)
        except:
            payoff = 0
        print(y[0],"\t",payoff, "\t",str(round(payoff/a.CLOSE[-1]*100,2))+"%" )
        logger.info(y[0]+": "+str(payoff)+" ("+str(round(payoff/a.CLOSE[-1]*100,2))+"%)")
    
        cumsum = cumsum + round(payoff/a.CLOSE[-1]*100,2)
        if portfolio.shape[0] == 1:
            a.plot(x, 360, False)
            pass
            
        #break
    
    

TATAMOTORS 	 -3.5 	 -1.62%


This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x1,y2 ]
[ (3,1) x1,y3 ]
[ (4,1) x1,y4 ]



In [53]:
print(cumsum)

8.41


In [38]:
a.buy[a.buy["buy"]]

,buy,low,close
date,,,


In [116]:
gc.get_count()

(359, 1, 0)

In [115]:
gc.collect()

100

In [8]:
tradelogger.setLevel(logging.INFO)
if mode == "Algo":
    #portfolio = downloadlist
    portfolio = ['BHARTIARTL']
    toDate = dt.datetime(2019,4,24,15,15)
    fromDate = getFromDate(toDate)
    STATE = ""
    stoploss = 0
    target = 0
    openpositions = 0
    portfolio = pd.DataFrame(portfolio)
    papertrade = True
    backtest(portfolio, fromDate, toDate)

BHARTIARTL: -5.8, (-1.7918 %)


- Lunch Time: 11:00 am to 2:00 pm => Choppy Market
- Bollinger band ZONE: for breakout

Choppy market Trading:
- Identify stocks with highest SD and trade using bollinger

Trending Market Trading:
- MACD or price action trading based on candle stick


No new open position after 3:00. Only close open positions

before 9:15: Go long based on one day candle
9:15 - 11:30: MACD or Stochastic+BB: One trigger, and atleast one confirmation
11:30 - 2:00: No trade untill and unless there is a golden cross or death cross
2:00 - 2:45: Go long if price is smaller than VWAP, Go short if price is higher than VWAP
2:00 - 2:45: Go long or short only if price crosses SMA(200)
